# PostgreSQL database preparation

Prerequisites:

1. You need to have PostgreSQL database installed and configured
2. You have downloaded the old but more comprehensive IMDB data set from year 2017. See the instructions and link to the files from [cinemagoer](https://cinemagoer.readthedocs.io/en/latest/usage/ptdf.html).
3. Create an empty database (imdb2017) for the user (postgres) with the following code:

```
CREATE DATABASE imdb2017
WITH OWNER postgres
TEMPLATE = template0
ENCODING UTF8
LC_COLLATE = 'und-x-icu'
LC_CTYPE = 'und-x-icu';
```

IMDB is providing newer but less comprehensive [dataset](https://www.imdb.com/interfaces/). Cinemagoer describes how to upload that data into Postgres but because the join order benchmark queries are built on the older dataset, we also need to use the older one.

In [1]:
# Database credentials

port = "5432"
pg_db_name = "imdb2017"
pg_user = "postgres"
pg_pw = "0000"
imdb_file_path = "C://Users//valte//Documents//frozendata"

pg_connection = "postgresql://" + pg_user + ":" + pg_pw + "@localhost:" + port + "/" + pg_db_name

The following code will use `imdbpy2sql` code to insert the internet movie database data into the created database. Note that the execution will probably take hours. Once the insert has been performed, comment the cell.

In [2]:
#!python cinemagoer\bin\imdbpy2sql.py -d {imdb_file_path} -u {pg_connection}

To speedup queries we create certain indeces.

In [3]:
import psycopg2

create_indices = """
create index company_id_movie_companies on movie_companies(company_id);
create index company_type_id_movie_companies on movie_companies(company_type_id);
create index info_type_id_movie_info on movie_info(info_type_id);
create index info_type_id_person_info on person_info(info_type_id);
create index keyword_id_movie_keyword on movie_keyword(keyword_id);
create index kind_id_aka_title on aka_title(kind_id);
create index kind_id_title on title(kind_id);
create index linked_movie_id_movie_link on movie_link(linked_movie_id);
create index link_type_id_movie_link on movie_link(link_type_id);
create index movie_id_aka_title on aka_title(movie_id);
create index movie_id_cast_info on cast_info(movie_id);
create index movie_id_complete_cast on complete_cast(movie_id);
create index movie_id_movie_companies on movie_companies(movie_id);
create index movie_id_movie_keyword on movie_keyword(movie_id);
create index movie_id_movie_link on movie_link(movie_id);
create index movie_id_movie_info on movie_info(movie_id);
create index person_id_aka_name on aka_name(person_id);
create index person_id_cast_info on cast_info(person_id);
create index person_id_person_info on person_info(person_id);
create index person_role_id_cast_info on cast_info(person_role_id);
create index role_id_cast_info on cast_info(role_id);
"""

test_query = "EXPLAIN ANALYZE SELECT * FROM title LIMIT 2;"

try:
    connection = psycopg2.connect(user=pg_user, password=pg_pw, host="localhost", port=port, database=pg_db_name)
    cursor = connection.cursor()
    cursor.execute(create_indices)
    res = cursor.fetchall()
    print(res)

except (Exception, psycopg2.Error) as error:
    print("Error while fetching data from PostgreSQL", error)

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

Error while fetching data from PostgreSQL relation "company_id_movie_companies" already exists

PostgreSQL connection is closed
